In [1]:
import pandas as pd

In [ ]:
# open each result file
# try to parse json result
# check if columns are correct
# create a table with term type, term, normalized term

In [2]:
# Read JSON file
dir_name = 'mistral_results_section_medmentions'

In [3]:
start_token = '<s> [INST] '
end_token = '[/INST]'
prompt = 'You are a medical AI expert. Your task is to create a list of all clinical, biomedical, chemical, or other UMLS concepts mentioned in the text. \nText: "'

In [33]:
import os

file_contents = []
for file in os.listdir(dir_name):
    filename = os.fsdecode(file)
    #filename = 'es-S0004-06142007000900010-10.txt'
    if filename.endswith(".txt"):
        with open(f'{dir_name}/{filename}') as file:
            text = file.read()            
            start_response = text.index(end_token)+len(end_token)
            
            is_complete = True
            if '</s>' in text[start_response:]:
                end_response = text[start_response:].index('</s>')
            else: 
                end_response = len(text[start_response:])-1
                is_complete = False
            
            start_request = len(start_token) + len(prompt) - 1
            end_request = start_request+ text[start_request:].index('"\n')
            request = text[start_request: end_request].strip()
            response = text[start_response:start_response+end_response].strip()
            file_contents.append({
                'filename': filename,
                'request': request,
                'response': response,
                'is_complete': is_complete
            })
    else:
        continue

In [35]:
file_contents[2597]

{'filename': '28487667_1671.txt',
 'request': 'No significant differences were found across mean-of-rule scorings. Best-of-rule and mean-of-rule scorings were significantly different except for the best-of-two vs. mean-of-two. The mean difference between highest and lowest rank across players was 6.7 (3.6), with individual rankings within the group varying 33% on average across procedures. One player moved from 3rd to 23rd place because of procedural differences.',
 'response': 'Based on the provided text, here is the list of concepts mentioned and their corresponding UMLS concept standard names:\n\n```json\n[]\n```\n\nThe text does not contain any clinical, biomedical, chemical, or other UMLS concepts mentioned explicitly. The numbers and percentages mentioned are descriptive statistics and do not correspond to specific UMLS concepts.',
 'is_complete': True}

In [36]:
df_file_contents = pd.DataFrame.from_records(file_contents)
df_file_contents.head()

,filename,request,response,is_complete
0,25847295_0.txt,"""Nonylphenol diethoxylate inhibits apoptosis i...","[\n{\n""original_concept"": ""Nonylphenol diethox...",True
1,25847295_1216.txt,"""The difference in structure of the two compou...","Based on the provided text, the following conc...",True
2,25847295_514.txt,"""Nonylphenol diethoxylate itself showed no tox...","[\n{\n""original_concept"": ""apoptosis"",\n""conce...",True
3,26876532_0.txt,"""EZH2 and ZFX oncogenes in malignant behaviour...","[\n{\n""original_concept"": ""EZH2"",\n""concept_st...",True
4,26876532_1017.txt,"""Two polymorphisms of the EZH2 gene were ident...","[\n{\n""original_concept"": ""EZH2 gene"",\n""conce...",True


In [37]:
df_file_contents.to_csv('mistral_file_contents_medmentions.tsv', sep='\t', index=False)

In [ ]:
# fix - search for ^""\w*\s\w* and replace with valid json, remove empty {}
# original_conconcept[\w_]*:
# original_conconcept[\w_]*""
# ,\n""original_concept" -> add opening {
# [^,]\n""concept -> add comma
# }\n{ - add comma
# "",\n} - remove trailing comma of last property
# _: to ""
# ""original_concept"", - no orig concept string
# ":_ -> concept starting a few characters later
# empty concept_standard_name - use the concept name

In [156]:
df_file_contents = pd.read_csv('mistral_file_contents_medmentions-fix.tsv', sep='\t')

In [157]:
df_file_contents[~df_file_contents['is_complete']]

,filename,request,response,is_complete


In [158]:
df_file_contents[~df_file_contents['response'].str.startswith('[')]

,filename,request,response,is_complete
1,25847295_1216.txt,"""The difference in structure of the two compou...","Based on the provided text, the following conc...",True
42,27241818_2901.txt,"""This regulatory mechanism limits the exercise...","Based on the provided text, here is the list o...",True
56,27252199_1432.txt,"""Together, our gross and histological examinat...","Based on the text provided, here is the list o...",True
67,27261587_1622.txt,"""The findings showed that combination therapy ...","Based on the text provided, here is the list o...",True
80,27273231_1434.txt,"""Hg in the acute phase, and persistent hemosta...","Based on the provided text, here is the list o...",True
...,...,...,...,...
2597,28487667_1671.txt,No significant differences were found across m...,"Based on the provided text, here is the list o...",True
2622,28502652_1650.txt,These results are a valuable contribution to u...,"Based on the text provided, there are no expli...",True
2637,28518001_1534.txt,Administration of beta blockers prior to satur...,"Based on the text provided, here is a list of ...",True
2655,28523846_1714.txt,ESI-IMS-MS is applicable as a process analytic...,"Based on the provided text, here is the list o...",True


In [159]:
df_file_contents[~df_file_contents['response'].str.endswith(']')]

,filename,request,response,is_complete
42,27241818_2901.txt,"""This regulatory mechanism limits the exercise...","Based on the provided text, here is the list o...",True
56,27252199_1432.txt,"""Together, our gross and histological examinat...","Based on the text provided, here is the list o...",True
67,27261587_1622.txt,"""The findings showed that combination therapy ...","Based on the text provided, here is the list o...",True
80,27273231_1434.txt,"""Hg in the acute phase, and persistent hemosta...","Based on the provided text, here is the list o...",True
105,27286174_1562.txt,"""A competing risk model conveys more relevant ...","[\n]\n\nThere are no specific clinical, biomed...",True
...,...,...,...,...
2622,28502652_1650.txt,These results are a valuable contribution to u...,"Based on the text provided, there are no expli...",True
2630,28508329_365.txt,Since patients with hepatocellular carcinoma (...,"[\n{\n""original_concept"": ""hepatocellular carc...",True
2637,28518001_1534.txt,Administration of beta blockers prior to satur...,"Based on the text provided, here is a list of ...",True
2655,28523846_1714.txt,ESI-IMS-MS is applicable as a process analytic...,"Based on the provided text, here is the list o...",True


In [160]:
df_file_contents.iloc[700]['response']

'[\n{\n"original_concept": "Food_Intake",\n"concept_standard_name": "Food intake"\n},\n{\n"original_concept": "Mean",\n"concept_standard_name": "Mean"\n},\n{\n"original_concept": "energy",\n"concept_standard_name": "Energy"\n},\n{\n"original_concept": "intake",\n"concept_standard_name": "Dietary intake"\n},\n{\n"original_concept": "infant",\n"concept_standard_name": "Infant"\n},\n{\n"original_concept": "girls",\n"concept_standard_name": "Female"\n},\n{\n"original_concept": "Al Ain",\n"concept_standard_name": "Al Ain"\n},\n{\n"original_concept": "Dubai",\n"concept_standard_name": "Dubai"\n},\n{\n"original_concept": "kcal",\n"concept_standard_name": "Calorie"\n},\n{\n"original_concept": "SD",\n"concept_standard_name": "Standard deviation"\n},\n{\n"original_concept": "toddlers",\n"concept_standard_name": "Toddler"\n},\n{\n"original_concept": "body mass index",\n"concept_standard_name": "Body mass index"\n},\n{\n"original_concept": "body weight",\n"concept_standard_name": "Body weight"\n},

In [161]:
import json
def parse_json(text):
    json_object = None
    text = text.strip()
    json_start = -1
    json_end = -1 #len(text)-1
    
    if '[' in text:
        json_start = text.index('[')
        if ']' in text:
            json_end = text.rindex(']')+1
        else:
            json_start = -1            
            
        #else:
            #print(text)
            # json incomplete
    if '{' in text and (text.index('{') < json_start or json_start == -1):
        json_start = text.index('{')
        json_end = text.rindex('}')+1
        
    substring = text[json_start:json_end]
    
    #if 't: "' in substring:
    #    substring = substring.replace('t: "', 't": "')
    
    try:
        #print(text[json_start:json_end])        
        if substring.startswith('{'):
            json_object = json.loads(f'[{substring}]')
        else:
            json_object = json.loads(substring)
    except ValueError as e:        
        #print(text)
        return None #rerun all not valid jsons
    return json_object

In [162]:
parse_json(df_file_contents.iloc[201]['response'])

In [194]:
df_file_contents['json'] = df_file_contents['response'].apply(parse_json)

In [195]:
df_file_contents.head()

,filename,request,response,is_complete,json
0,25847295_0.txt,"""Nonylphenol diethoxylate inhibits apoptosis i...","[\n{\n""original_concept"": ""Nonylphenol diethox...",True,[{'original_concept': 'Nonylphenol diethoxylat...
1,25847295_1216.txt,"""The difference in structure of the two compou...","Based on the provided text, the following conc...",True,"[{'original_concept': 'structure', 'concept_st..."
2,25847295_514.txt,"""Nonylphenol diethoxylate itself showed no tox...","[\n{\n""original_concept"": ""apoptosis"",\n""conce...",True,"[{'original_concept': 'apoptosis', 'concept_st..."
3,26876532_0.txt,"""EZH2 and ZFX oncogenes in malignant behaviour...","[\n{\n""original_concept"": ""EZH2"",\n""concept_st...",True,"[{'original_concept': 'EZH2', 'concept_standar..."
4,26876532_1017.txt,"""Two polymorphisms of the EZH2 gene were ident...","[\n{\n""original_concept"": ""EZH2 gene"",\n""conce...",True,"[{'original_concept': 'EZH2 gene', 'concept_st..."


In [165]:
df_file_contents[df_file_contents['json'].isnull()].shape

(31, 5)

In [166]:
df_file_contents[df_file_contents['json'].isnull()]

,filename,request,response,is_complete,json
201,27342634_1115.txt,"""Low-frequency high-intensity ultrasound cause...","[\n{\n""original_concept"": ""ultrasound"",\n""conc...",True,None
367,27427386_1076.txt,"""These cybrids cell lines bearing m.14502T > C...","Based on the text provided, here is the list o...",True,None
576,27503306_0.txt,"""Preclinical fibrinolysis in patients with ST-...","[\n{\n""original_concept"":ST-segment elevation ...",True,None
696,27552906_1796.txt,"""Calculated depositions confirmed appropriate ...","[\n{\n""original_concept"": ""Calculated depositi...",True,None
724,27572277_1312.txt,"""Most of the 12 discriminant genera correlated...","[\n{\n""original_concept"": ""discriminant genera...",True,None
846,27629431_0.txt,"""Suppressed translation and ULK1 degradation a...","[\n{\n""original_concept"": ""autophagy"",\n""conce...",True,None
929,27659602_0.txt,"""The Aortic Root: Natural History After Root-S...","[\n{\n""original_concept"": root-sparing aortic ...",True,None
962,27671177_483.txt,"""Atotal of 349 children aged 12-year from 14 r...","[\n{\n""original_concept"": ""children"",\n""concep...",True,None
1152,27779702_2014.txt,"""Collectively, through drug efficacy identific...","Based on the given text, here's the list of co...",True,None
1182,27791362_741.txt,"""We show that these PEG-PLA - encapsulated CWO...","[\n{\n""original_concept"": PEG-PLA"",\n""concept_...",True,None


In [167]:
df_file_contents.shape

(2690, 5)

In [207]:
responses = []

for index, row in df_file_contents.iterrows():
    obj_type = type(row['json'])
    if obj_type is list:
        for item in row['json']:
            if type(item) is dict:
                                
                if 'conventional_concept_name' in item.keys():
                    item['concept_standard_name'] = item['conventional_concept_name']
                
                for key in list(item.keys()):
                    if key not in ['original_concept', 'concept_standard_name', 'filename']:
                        item['original_concept'] = item[key]
                #if item.has_key('origincal_concept'):
                #    item['original_concept'] = item['origincal_concept']
                    #original_concentration, original_convergence, original_concept_exercise, original_concept_CBT, 
                    #original_concept_TDR, original_concept_chronic, original_concept_LBP, original_concept_patients
                
                if 'concept_standard_name' not in item.keys() or item['concept_standard_name'] == '' or item['concept_standard_name'] == None:
                    item['concept_standard_name'] = item['original_concept']
                
                item['filename'] = row['filename']
                responses.append(item)
            else:
                item_dict = {}                
                item_dict['original_concept'] = item
                item_dict['concept_standard_name'] = item
                item_dict['filename'] = row['filename']
                responses.append(item_dict)
    #elif obj_type is dict:
    #    print(row['json'])
    #    for item in row['json']['terms']:
    #        item['filename'] = row['filename']
    #        responses.append(item)

In [208]:
df_response = pd.DataFrame.from_records(responses)

In [209]:
df_response.head()

,original_concept,concept_standard_name,filename,conventional_concept_name,original_concentration,original_convergence,original_concept_exercise,original_concept_CBT,original_concept_TDR,original_concept_chronic,original_concept_LBP,original_concept_patients
0,Nonylphenol diethoxylate,Nonylphenol diethoxylate,25847295_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,apoptosis,Apoptosis,25847295_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PC12 cells,PC12 Cell Line,25847295_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aquatic environment,Aquatic Environment,25847295_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,wastewater contaminants,Wastewater Contaminants,25847295_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
df_response[df_response['concept_standard_name']=='']

,original_concept,concept_standard_name,filename,conventional_concept_name,original_concentration,original_convergence,original_concept_exercise,original_concept_CBT,original_concept_TDR,original_concept_chronic,original_concept_LBP,original_concept_patients


In [212]:
import re
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return ' '.join([m.group(0) for m in matches])

In [213]:
def normalize_type(term_type):
    term_type = term_type.replace('_', ' ')
    term_type = camel_case_split(term_type)
    
    return term_type.lower()

In [214]:
normalize_type('MedicalProcedure')

'medical procedure'

In [215]:
df_response['normalized_type'] = df_response['concept_standard_name'].apply(normalize_type)

In [217]:
df_response[df_response['normalized_type']!=df_response['concept_standard_name'].str.lower()]

,original_concept,concept_standard_name,filename,conventional_concept_name,original_concentration,original_convergence,original_concept_exercise,original_concept_CBT,original_concept_TDR,original_concept_chronic,original_concept_LBP,original_concept_patients,normalized_type
630,mRNA stimulator,"mRNA, stability",27257056_590.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"m rna, stability"
891,Dendritic cells,Dendritic cells (DCs),27273307_480.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dendritic cells (d cs)
1208,HeLa cells,HeLa cell line,27297967_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he la cell line
1214,HeLa,HeLa cells,27297967_1181.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he la cells
1236,HeLa cells,HeLa cells,27297967_754.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he la cells
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25693,Long Non-Coding RNAs,Long non-coding RNAs,28508329_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,long non-coding rn as
25852,FerMEG12,FerMEG12,28521168_0.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fer meg12
26036,Orb2A mRNA,Orb2A mRNA,28525754_462.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,orb2a m rna
26353,GTPase-activating proteins,GTPase-activating proteins,28550165_896.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gt pase-activating proteins


In [218]:
len(list(df_response['normalized_type'].unique()))

17975

In [219]:
df_response.to_csv('mistral_medmentions_mentions.tsv', sep='\t', index=False)

In [220]:
df_response_new = df_response[['filename', 'original_concept', 'concept_standard_name']]

In [ ]:
# find mapping in original text for each term

In [221]:
df_file_contents_idx = df_file_contents.set_index('filename')

In [222]:
df_file_contents_idx.loc['27273307_480.txt']['request']

'"Human chorionic gonadotropin was previously thought to be necessary for OHSS to occur; however, recent case reports have proven otherwise. The contribution of an attenuated anti-Mullerian hormone signalling pathway and CD11c + HLA-DR + dendritic cells and associated interleukins has been explored recently as contributors to pathogenesis. Treatment is largely supportive and is based mainly on consensus statements rather than evidence. Therefore, it is important to prevent this condition by identifying women at risk, allowing the clinician to implement preventive strategies, including the use of GnRH antagonist cycles with agonist triggers.'

In [235]:
def find_matches(mention, text):
    results = [] #mention, start, end
    #print(mention, text)
    matches = re.finditer(re.escape(mention), text)
    return [(m.start(0), m.end(0)) for m in matches]

In [233]:
find_matches('dendritic cells'.lower(), df_file_contents_idx.loc['27273307_480.txt']['request'])

dendritic cells "Human chorionic gonadotropin was previously thought to be necessary for OHSS to occur; however, recent case reports have proven otherwise. The contribution of an attenuated anti-Mullerian hormone signalling pathway and CD11c + HLA-DR + dendritic cells and associated interleukins has been explored recently as contributors to pathogenesis. Treatment is largely supportive and is based mainly on consensus statements rather than evidence. Therefore, it is important to prevent this condition by identifying women at risk, allowing the clinician to implement preventive strategies, including the use of GnRH antagonist cycles with agonist triggers.


[(237, 252)]

In [ ]:
# foreach find all matches
# find nonoverlapping
# save

In [242]:
all_matches = []
for index, row in df_file_contents.iterrows():
    text = row['request']
    if text.startswith('"'):
        text = text[1:]
    mentions = df_response[df_response['filename'] == row['filename']]
    
    for midx, mrow in mentions.iterrows():
        term = mrow['original_concept'].lower()
        matches = find_matches(term, text.lower())
        
        for match in matches:
            all_matches.append({
                'filename': row['filename'],
                'term': mrow['original_concept'],
                'start': match[0],
                'end': match[1],
                #'type': mrow['type'],
                #'normalized_type': mrow['normalized_type'],
                'normalized_term': mrow['concept_standard_name']
            })

In [243]:
df_all_matches = pd.DataFrame.from_records(all_matches)
df_all_matches.head()

,filename,term,start,end,normalized_term
0,25847295_0.txt,Nonylphenol diethoxylate,0,24,Nonylphenol diethoxylate
1,25847295_0.txt,Nonylphenol diethoxylate,370,394,Nonylphenol diethoxylate
2,25847295_0.txt,apoptosis,34,43,Apoptosis
3,25847295_0.txt,apoptosis,259,268,Apoptosis
4,25847295_0.txt,apoptosis,437,446,Apoptosis


In [244]:
df_all_matches['filename'] = df_all_matches['filename'].apply(lambda x: x.replace('.txt', ''))

In [245]:
df_all_matches.shape

(33715, 5)

In [246]:
df_all_matches.to_csv('all_matches_medmentions.tsv', sep='\t', index=False)

In [ ]:
#transform from section to full file

In [155]:
df_all_matches['note_id'] = df_all_matches['filename']
df_all_matches['concept_id']=1
df_all_matches[['filename','start','end','concept_id']].to_csv('pred_annotations_ner_medmentions.csv', index=False)